<a href="https://colab.research.google.com/github/GemmaGorey/Dissertation/blob/main/Audio_Processing_V2_Dissertation_GG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initial Colab setup below - Run  top two cells once only per session**


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
# install mamba to use instead of pip

In [ ]:
# Create the config file and build the environment.
yaml_content = """
name: dissertation
channels:
  - pytorch
  - conda-forge
dependencies:
  - python=3.11
  - pytorch=2.2.2
  - torchvision=0.17.2
  - torchaudio
  - librosa
  - numpy<2
  - pandas
  - jupyter
  - wandb
"""

# Write the string content to a file -  'environment.yml'.
with open('environment.yml', 'w') as f:
    f.write(yaml_content)

print("environment.yml file created successfully.")

# create the environment using mamba from the yml file.
print("\n Creating environment")

!mamba env create -f environment.yml --quiet && echo -e "\n 'dissertation' environment is ready to use."

In [ ]:
# imports and setting up of GitHub and W&B

# clone project repository from GitHub
print("⏳ Cloning GitHub repository...")
!git clone https://github.com/GemmaGorey/Dissertation.git
print("Repository cloned.")

#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#imports
import pandas as pd
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
from transformers import AutoTokenizer
from tqdm.auto import tqdm
import torch
import glob

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') #loading the tokenizer for lyrics processing
print("Tokenizer loaded.")

In [ ]:
#loading in the dataset -using complete which has the lyrics and music.

# create string for path to where dataset lives
data_path = '/content/drive/MyDrive/dissertation/audio/'

# load the files

print("Loading MERGE Metadata ")
merge_df = pd.read_csv(data_path + 'merge_bimodal_complete_metadata.csv')

print("\n Loading Valence-Arousal values")
av_df = pd.read_csv(data_path + 'merge_bimodal_complete_av_values.csv')


print("\n Datasets loaded successfully.")

# Inspect the files
print("\n First 5 rows of the data")
display(merge_df.head())

print("\n First 5 rows of the MERGE AV Values")
display(av_df.head())

In [ ]:
# merging the two dataframes using the common column

final_df = pd.merge(merge_df, av_df, left_on='Audio_Song', right_on='Audio_Song')

print(" DataFrames merged")

print("\n First 5 rows  MASTER DataFrame")
display(final_df.head())

In [ ]:
#checking data - in different quadrants
print(final_df['Quadrant'].value_counts())

In [ ]:
#checking data
print(final_df[['Valence', 'Arousal']].describe())

In [ ]:
#checking no blank entries
print(final_df[['Quadrant', 'Valence', 'Arousal']].info())

In [ ]:
#global parameters for the audio processing

TARGET_SR = 22050 #used in the AllMusic dataset, which is where this original data is from.(may try 16k to see if it lowers accuracy)
N_FFT = 2048#  using Librosa's (for music) standard values but can change to test
HOP_LENGTH = 512 # using Librosa's (for music) standard values but can change to test
N_MELS = 128 # using Librosa's (for music) standard values but can change to test

def find_global_max_power(dataframe, base_audio_path):
    """
    Loops through all audio files in the audio dataframe to find the
    single maximum power value in any Mel spectrogram. This is used as a form of normalisation
    """
    print("Finding global max")
    # Start off with a very small number
    global_max_power = -np.inf

    # Use tqdm for a progress bar
    for index, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]):
        try:
            song_id = row['Audio_Song']
            quadrant = row['Quadrant']

            # Construct the audio file path
            audio_path = os.path.join(base_audio_path, quadrant, f"{song_id}.mp3")

            # Load audio
            y, sr = librosa.load(audio_path, sr=TARGET_SR)

            # Get spectrogram using the global variables
            mel_spec_power = librosa.feature.melspectrogram(
                y=y,
                sr=sr,
                n_fft=N_FFT,
                hop_length=HOP_LENGTH,
                n_mels=N_MELS
            )

            # Find max in each one
            current_max_power = np.max(mel_spec_power)

            # Update global max if this is higher
            if current_max_power > global_max_power:
                global_max_power = current_max_power

        except Exception as e:
            print(f"\nSkipping file {song_id} due to error: {e}")

    print("\n Global max found")
    return global_max_power




In [ ]:
GLOBAL_MAX = find_global_max_power(final_df, data_path)

In [ ]:
print(GLOBAL_MAX)

In [ ]:
GLOBAL_MAX = 13400.846

#Output from before so don't have to run each time - This is the power output max

In [ ]:
def find_global_max_and_outlier(dataframe, base_audio_path):
    """
    Loops through all audio files to find the single maximum power value
    and prints the song ID responsible for each new maximum.
    """

    # Start off with a very small number
    global_max_power = -np.inf
    outlier_file = "None"
    outlier_quadrant = "None"

    # Use tqdm for a progress bar
    for index, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]):
        try:
            song_id = row['Audio_Song']
            quadrant = row['Quadrant']

            # Construct the audio file path
            audio_path = os.path.join(base_audio_path, quadrant, f"{song_id}.mp3")

            # Load audio
            y, sr = librosa.load(audio_path, sr=TARGET_SR)

            # Check for silent/corrupt files
            if np.max(np.abs(y)) < 0.001:
                # This skips (near) silent files, which can also be a problem
                continue

            # Get spectrogram using the global variables
            mel_spec_power = librosa.feature.melspectrogram(
                y=y,
                sr=sr,
                n_fft=N_FFT,
                hop_length=HOP_LENGTH,
                n_mels=N_MELS
            )

            # Find max in each one
            current_max_power = np.max(mel_spec_power)

            # Update global max if this is higher
            if current_max_power > global_max_power:
                global_max_power = current_max_power
                outlier_file = song_id
                outlier_quadrant = quadrant
                print(f"\nNew Max Found: {global_max_power:.2f}")
                print(f"   (From Song ID: {outlier_file} in Quadrant: {outlier_quadrant})")

        except Exception as e:
            # This will also catch corrupt files that librosa can't even load
            print(f"\nSKIPPING FILE {song_id} due to error: {e}")

    print("\nComplete")
    print(f"The HIGHEST max power was: {global_max_power:.2f}")
    print(f"This value came from the file: {outlier_file} (Quadrant: {outlier_quadrant})")

    return global_max_power, outlier_file

    #DECIDED OFF THE BACK OF THIS EXPERIMENT NOT TO USE THE GLOBAL MAX. PUSHES THE SCALE TO NEGATIVE WHICH IS NEARER THE FLOOR OF -80.

In [ ]:
GLOBAL_MAX_OUTLIER = find_global_max_and_outlier(final_df, data_path)

In [ ]:
def load_song_data(song_id, lyric_id, quadrant):
    """
    Loads the audio and lyrics for a given song ID
    """
    print(f"Attempting to load song: {song_id}")
    try:
        # Construct the file path to google drive
        base_path = '/content/drive/MyDrive/dissertation/'

        # Audio files url addition as these are in subfolders for each emotion quadrant
        audio_path = os.path.join(base_path, 'audio', quadrant, f"{song_id}.mp3")

        # Lyric files url addition as these are in a separate main folder
        lyrics_path = os.path.join(base_path, 'lyrics', quadrant, f"{lyric_id}.txt")

        # load the audio file
        audio_waveform, sample_rate = librosa.load(audio_path, sr=TARGET_SR)

        #Process Audio into a Mel Spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=audio_waveform, sr=sample_rate, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS)

        # Convert to decibels
        db_spectrogram = librosa.power_to_db(mel_spectrogram, ref=GLOBAL_MAX)

        target_length = 1292  # A fixed length for all spectrograms, so same length tensors later - chosen for 30 seconds, and sr of 22050, and hop_length of 512
        current_length = db_spectrogram.shape[1]

        if current_length > target_length:
            # If it is too long, truncate it
            db_spectrogram = db_spectrogram[:, :target_length]
        elif current_length < target_length:
            # If too short, pad it
            padding_needed = target_length - current_length
            db_spectrogram = np.pad(db_spectrogram,
                                    ((0, 0), (0, padding_needed)),
                                    mode='constant',
                                    constant_values=-80.0) #close to silence so doesn't impact

        # load lyrics text
        with open(lyrics_path, 'r', encoding='utf-8') as f:
            lyrics_text = f.read()

        #Tokenise raw text
        encoded_lyrics = tokenizer(
            lyrics_text,
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )

        print(f"Successfully loaded and processed {song_id}")
        #returns the spectrogram and the tokenised lyrics
        return db_spectrogram, encoded_lyrics

    except Exception as e:
        print(f"An error occurred for {song_id}: {e}")
        return None, None


In [ ]:

#testing the output of the above function by picking a song.

#pick a song between 0 and 2215
test_song_index = 1111

test_audio_id = final_df.iloc[test_song_index]['Audio_Song']
test_lyric_id = final_df.iloc[test_song_index]['Lyric_Song_x']
test_quadrant = final_df.iloc[test_song_index]['Quadrant']

print(f"Audio ID: {test_audio_id}")
print(f"Lyric ID: {test_lyric_id}")
print(f"Quadrant: {test_quadrant}")

#variables for the spectorgram and tokenised lyrics loaded.
spectrogram, encoded_lyrics = load_song_data(test_audio_id, test_lyric_id, test_quadrant)


#check the output of the lyrics
if encoded_lyrics is not None:
    print("\n--- Tokenized Lyrics Output ---")

    #dictionary of tensors
    print("Encoded Tensors:")
    display(encoded_lyrics)

    #readable tokens to check
    tokens = tokenizer.convert_ids_to_tokens(encoded_lyrics['input_ids'][0])

    print("\nSample of Tokens Produced:")
    print(tokens[:50])

#check the output of the audio
if spectrogram is not None:
    print("\n--- Spectrogram Data ---")
    print(f"Shape: {spectrogram.shape}")

    #show spectogram
    print("\nDisplaying Spectrogram:")
    plt.figure(figsize=(12, 4))
    #get the sample rate for the axis display
    audio_file_path = os.path.join('/content/drive/MyDrive/dissertation/audio', test_quadrant, f'{test_audio_id}.mp3')
    sr = librosa.get_samplerate(path=audio_file_path)

    librosa.display.specshow(spectrogram, sr=sr, x_axis='time', y_axis='mel')

    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Mel Spectrogram for {test_audio_id}')
    plt.tight_layout()
    plt.show()

In [ ]:

#define where to save the files
base_path = '/content/drive/MyDrive/dissertation/merge_dataset/output_from_code/'
spectrogram_save_dir = os.path.join(base_path, 'processed_spectrograms/')
lyrics_save_dir = os.path.join(base_path, 'processed_lyrics/')

# Create the directories if they don't exist
os.makedirs(spectrogram_save_dir, exist_ok=True)
os.makedirs(lyrics_save_dir, exist_ok=True)

print(f"Spectrograms will be saved in: {spectrogram_save_dir}")
print(f"Lyric tensors will be saved in: {lyrics_save_dir}")

#process the audio and lyrics storing info as processed
processed_records = []

#iterate over all rows and get origress bar
for index, row in tqdm(final_df.iterrows(), total=final_df.shape[0], desc="Processing Songs"):

    #get song info from each row
    song_id = row['Audio_Song']
    lyric_id = row['Lyric_Song_x']
    quadrant = row['Quadrant']

    #call function to transform
    spectrogram, encoded_lyrics = load_song_data(song_id, lyric_id, quadrant)

    #check if it worked
    if spectrogram is not None and encoded_lyrics is not None:
        try:
            #save the spectorgram
            spectrogram_path = os.path.join(spectrogram_save_dir, f"{song_id}.npy")
            np.save(spectrogram_path, spectrogram)

            #save tokenised lyric tensors
            lyrics_path = os.path.join(lyrics_save_dir, f"{song_id}.pt")
            torch.save(encoded_lyrics, lyrics_path)

            #record file path
            processed_records.append({
                'song_id': song_id,
                'spectrogram_path': spectrogram_path,
                'lyrics_path': lyrics_path,
                'valence': row['Valence'],
                'arousal': row['Arousal'],
                'quadrant': quadrant
            })
        except Exception as e:
            print(f"Error saving files for {song_id}: {e}")


#save new dataframe for using in experiments
processed_df = pd.DataFrame(processed_records)
master_file_path = os.path.join(base_path, 'master_processed_file_list.csv')
processed_df.to_csv(master_file_path, index=False)

print("\n PROCESSING COMPLETE!")
print(f"Saved {len(processed_df)} records.")
print(f"Master file list saved to: {master_file_path}")
display(processed_df.head())

In [ ]:

#path to where this was saved
base_path = '/content/drive/MyDrive/dissertation/merge_dataset/output_from_code/'
master_file_path = os.path.join(base_path, 'master_processed_file_list.csv')

#load the master csv with all the paths and VA values
master_df = pd.read_csv(master_file_path)

print("Master csv loaded")
display(master_df.head())

#check data for 1 song
#pick a song between 0 and 2215
test_final_song_index = 1111
song_info = master_df.iloc[test_final_song_index]

print(f"\nLoading data for song: {song_info['song_id']}")

#Load the spectrogram from the file
spectrogram = np.load(song_info['spectrogram_path'])

#load the lyric tensors
encoded_lyrics = torch.load(song_info['lyrics_path'], weights_only=False)

#Get the labels
valence = song_info['valence']
arousal = song_info['arousal']

#check the data
print("Spectrogram Shape:", spectrogram.shape)
print("Encoded Lyrics Tensors:", encoded_lyrics)
print(f"Labels - Valence: {valence}, Arousal: {arousal}")


In [ ]:

import math


# Load Master CSV File
print("Loading Master CSV...")
base_path = '/content/drive/MyDrive/dissertation/merge_dataset/output_from_code/'
master_file_path = os.path.join(base_path, 'master_processed_file_list.csv')

try:
    master_df = pd.read_csv(master_file_path)
    print(f"Successfully loaded {len(master_df)} file records.")
except FileNotFoundError:
    print(f"ERROR: Could not find master file at {master_file_path}")
    master_df = pd.DataFrame(columns=['song_id', 'spectrogram_path'])

if not master_df.empty:
    BATCH_SIZE = 24
    COLS = 6
    ROWS_PER_BATCH = math.ceil(BATCH_SIZE / COLS) # This will be 4

    num_batches = math.ceil(len(master_df) / BATCH_SIZE)

    print(f"Total items: {len(master_df)}. Batch size: {BATCH_SIZE}. Total batches: {num_batches}.")

    for batch_num in range(num_batches):
        print("-" * 50)
        print(f"Displaying Batch {batch_num + 1} of {num_batches}")

        start_index = batch_num * BATCH_SIZE
        end_index = min((batch_num + 1) * BATCH_SIZE, len(master_df))
        batch_df = master_df.iloc[start_index:end_index]

        fig, axs = plt.subplots(ROWS_PER_BATCH, COLS,
                                figsize=(COLS * 3.5, ROWS_PER_BATCH * 3),
                                constrained_layout=True)

        if not isinstance(axs, np.ndarray):
            axs = np.array([axs])
        axs_flat = axs.flatten()

        img = None

        for index, (master_idx, row) in enumerate(batch_df.iterrows()):
            ax = axs_flat[index]
            song_id = row['song_id']
            spec_path = row['spectrogram_path']

            try:
                spectrogram = np.load(spec_path)

                img = librosa.display.specshow(
                    spectrogram,
                    sr=TARGET_SR,
                    hop_length=HOP_LENGTH,
                    x_axis='time',
                    y_axis='mel',
                    ax=ax
                )

                ax.set_title(song_id, fontsize=8)

            except Exception as e:
                ax.set_title(f"Error loading {song_id}", fontsize=8)
                ax.text(0.5, 0.5, str(e), ha='center', va='center', wrap=True, fontsize=6, color='red')

            ax.set_xlabel(None)
            ax.set_ylabel(None)

        num_items_in_batch = len(batch_df)
        for i in range(num_items_in_batch, len(axs_flat)):
            axs_flat[i].axis('off')

        if img is not None:
            fig.colorbar(img, ax=axs.ravel().tolist(), format='%+2.0f dB', shrink=0.8)

        plt.show()

else:
    print("DataFrame is empty. No plots to generate.")
